## Import

In [ ]:
%load_ext autoreload
%autoreload 2
import logging

import seaborn as sns

import core.config as cfg
import core.finance as fin
import helpers.dbg as dbg
import helpers.env as env
import helpers.printing as pri
import vendors.kibot.utils as kut

In [ ]:
print(env.get_system_signature())

pri.config_notebook()

# TODO(gp): Changing level during the notebook execution doesn't work. Fix it.
# dbg.init_logger(verb=logging.DEBUG)
dbg.init_logger(verb=logging.INFO)
# dbg.test_logger()

_LOG = logging.getLogger(__name__)

# Metadata

In [ ]:
df1 = kut.read_metadata1()
df1.head(3)

In [ ]:
df2 = kut.read_metadata2()
df2.head(3)

In [ ]:
df3 = kut.read_metadata3()
df3.head(3)

In [ ]:
df4 = kut.read_metadata4()
print(df4.head(3))

print(df4["Exchange"].unique())

## Explore metadata

In [ ]:
mask = ["GAS" in d or "OIL" in d for d in df4["Description"]]
print(sum(mask))
print(df4[mask].drop(["SymbolBase", "Size(MB)"], axis=1))

# Read data

In [ ]:
config = cfg.Config.from_env()

if config is None:
    config = cfg.Config()
    config_tmp = config.add_subconfig("read_data")
    # config_tmp["nrows"] = 100000
    config_tmp["nrows"] = None
    #
    config["zscore_com"] = 28

print(config)

# Prices

## Read daily prices

In [ ]:
symbols = "CL NG RB BZ".split()
file_name = (
    "s3://alphamatic/kibot/All_Futures_Continuous_Contracts_daily/%s.csv.gz"
)

daily_price_dict_df = kut.read_multiple_symbol_data(
    symbols, file_name, nrows=config["read_data"]["nrows"]
)

daily_price_dict_df["CL"].tail(2)

In [ ]:
col_name = "close"
mode = "pct_change"
daily_rets = kut.compute_ret_0_from_multiple_daily_prices(
    daily_price_dict_df, col_name, mode
)

daily_rets.head(3)

In [ ]:
daily_rets = daily_rets["2008-01-01":]
daily_rets.fillna(0.0).cumsum().plot()

In [ ]:
daily_zrets = fin.zscore(
    daily_rets, com=config["zscore_com"], demean=False, standardize=True, delay=1
)
daily_zrets.columns = [c.replace("ret_", "zret_") for c in daily_zrets.columns]
daily_zrets.dropna().head(3)

In [ ]:
daily_zrets = daily_zrets["2008-01-01":]

daily_zrets.fillna(0.0).cumsum().plot()

In [ ]:
annot = True
stocks_corr = daily_zrets.corr()

sns.clustermap(stocks_corr, annot=annot)

In [ ]:
print(daily_zrets.head(2))
daily_rets.to_csv("oil_daily_zrets.csv")

## Read 1 min prices

In [ ]:
symbols = "CL NG RB BZ".split()
file_name = (
    "s3://alphamatic/kibot/All_Futures_Continuous_Contracts_1min/%s.csv.gz"
)
min_price_dict_df = kut.read_multiple_symbol_data(
    symbols, file_name, nrows=config["read_data"]["nrows"]
)

min_price_dict_df["CL"].tail(2)

In [ ]:
min_price_dict_df.keys()

In [ ]:
mode = "pct_change"
min_rets = kut.compute_ret_0_from_multiple_1min_prices(min_price_dict_df, mode)

min_rets.head(3)

In [ ]:
min_rets.fillna(0.0).resample("1D").sum().cumsum().plot()

In [ ]:
min_zrets = fin.zscore(
    min_rets, com=config["zscore_com"], demean=False, standardize=True, delay=1
)
min_zrets.columns = [c.replace("ret_", "zret_") for c in min_zrets.columns]
min_zrets.dropna().head(3)

In [ ]:
min_zrets = min_rets["2008-01-01":]

min_zrets.fillna(0.0).resample("1d").sum().cumsum().plot()

In [ ]:
annot = True
stocks_corr = min_rets.corr()

sns.clustermap(stocks_corr, annot=annot)

In [ ]:
print(min_zrets.head(2))
min_zrets.to_csv("oil_1min_zrets.csv")